In [1]:
# import sagemaker SDK
import sagemaker

print(sagemaker.__version__)

2.75.1


In [1]:
# setup - install JDK
# you only need to run this once per KernelApp
%conda install openjdk -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /home/zqc/anaconda3/envs/airflow_env

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openjdk-8.0.332            |       h166bdaf_0        97.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        97.8 MB

The following NEW packages will be INSTALLED:

  openjdk            conda-forge/linux-64::openjdk-8.0.332-h166bdaf_0



openjdk-8.0.332      | 97.8 MB   | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...wo

In [6]:
# import PySpark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

Ivy Default Cache set to: /home/zqc/.ivy2/cache
The jars for the packages stored in: /home/zqc/.ivy2/jars
:: loading settings :: url = jar:file:/home/zqc/anaconda3/envs/airflow_env/lib/python3.7/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bc8e6f2d-3529-47d3-838b-b7ade2fcc8f3;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.563/aws-java-sdk-bundle-1.11.563.jar ...


Now that you have installed PySpark and initiated a Spark session, let's try out a couple of sample Pandas user defined functions (UDF).

In [7]:
from pyspark.sql.types import *
from pyspark.context import SparkContext

from pyspark.sql.functions import (
    col,
    count,
    rand,
    collect_list,
    explode,
    struct,
    count,
    lit,
)
from pyspark.sql.functions import pandas_udf, PandasUDFType

sc = SparkContext('local')
spark = SparkSession(sc)

# generate random data
df = (
    spark.range(0, 10 * 100 * 100)
    .withColumn("id", (col("id") / 100).cast("integer"))
    .withColumn("v", rand())
)
df.cache()
df.count()

df.show()

23/02/02 17:12:48 WARN Utils: Your hostname, zhangqinchuan resolves to a loopback address: 127.0.1.1; using 172.19.186.130 instead (on interface eth0)
23/02/02 17:12:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/02 17:12:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+---+--------------------+
| id|                   v|
+---+--------------------+
|  0| 0.34857940639844887|
|  0|  0.2088843626214535|
|  0|  0.9577188505061983|
|  0|  0.7372880435600724|
|  0|  0.2544530997135277|
|  0| 0.46724977436394455|
|  0|  0.5003725231636404|
|  0|  0.6921533114907326|
|  0| 0.38695771043189997|
|  0|   0.508820584322134|
|  0|  0.5095775991714272|
|  0| 0.31687758980157477|
|  0|  0.8122500198314806|
|  0|   0.688595791494441|
|  0|0.008732864399759732|
|  0| 0.43890907196679696|
|  0| 0.23843125300844337|
|  0| 0.23762814800889476|
|  0|  0.6335448406490326|
|  0|  0.5099400202934323|
+---+--------------------+
only showing top 20 rows



In [8]:
# sample pandas udf to return squared value
@pandas_udf("double", PandasUDFType.SCALAR)
def pandas_squared(v):
    return v * v


df.withColumn("v2", pandas_squared(df.v))

DataFrame[id: int, v: double, v2: double]

In [9]:
df2 = (
    df.withColumn("y", rand())
    .withColumn("x1", rand())
    .withColumn("x2", rand())
    .select("id", "y", "x1", "x2")
)
df2.show()

+---+-------------------+-------------------+-------------------+
| id|                  y|                 x1|                 x2|
+---+-------------------+-------------------+-------------------+
|  0| 0.9496667131158872|0.31657918118705486| 0.7840432647540505|
|  0|0.29494229648149173| 0.5885629332832106| 0.9091660853188566|
|  0| 0.5482080607263939| 0.1874023705387422| 0.5685358929673487|
|  0| 0.6007644849083354| 0.0394802673279353| 0.8216625579310718|
|  0|0.07871132374499978| 0.6590352280143387| 0.2976351871769567|
|  0|  0.545962237778807| 0.0756127303680253| 0.8041423922458693|
|  0| 0.0235432683535296| 0.1644863362855733|0.42421380939402764|
|  0| 0.5560372426946423|0.03445009987797776|0.21012616937087525|
|  0|0.26721589373035226|  0.959004579661844| 0.2840591670386182|
|  0| 0.2893542126215911| 0.1243963490622556|0.19768101374138214|
|  0| 0.4059060167459927| 0.9138279344860069| 0.9681735033828731|
|  0|0.05881735786624165|  0.629839262846955|  0.717064995534248|
|  0| 0.56

In [13]:
import pandas as pd
import statsmodels.api as sm

group_column = "id"
y_column = "y"
x_columns = ["x1", "x2"]
schema = df2.select(group_column, *x_columns).schema

ModuleNotFoundError: No module named 'statsmodels.api'